In [1]:

!pip install facenet-pytorch==2.5.2  # face detection + embeddings
!pip install torchvision
!pip install Pillow
!pip install mtcnn
!pip install matplotlib
!pip install pandas
!pip install scipy
!pip install scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 82.6 MB/s eta 0:00:00


In [2]:

import os, io, time, base64, pickle, uuid
from PIL import Image
import numpy as np
import pandas as pd
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import transforms
from IPython.display import display, Javascript, HTML
from google.colab import files

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

# face detector + embedder
mtcnn = MTCNN(keep_all=False, device=device)   # detect a single face per image
resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)  # embeddings

# path for DB
DB_PATH = 'face_db.pkl'   # simple pickle-based DB for prototype
ATTENDANCE_CSV = 'attendance_log.csv'


Device: cuda


  0%|          | 0.00/107M [00:00<?, ?B/s]

In [4]:

def img_from_bytes(bytestr):
    return Image.open(io.BytesIO(bytestr)).convert('RGB')

def get_face_tensor(img_pil):
    """
    Detects face and returns aligned face tensor for embedding,
    or None if no face detected.
    """
    # mtcnn returns PIL or torch tensor depending; use preprocess=True return tensor
    face = mtcnn(img_pil)  # returns a torch.Tensor (3x160x160) or None
    return face

def get_embedding(face_tensor):
    """
    face_tensor: torch.Tensor (3x160x160) or batch (n x 3 x 160 x 160)
    returns 512-d numpy vector (L2 normalized)
    """
    if face_tensor is None:
        return None
    if face_tensor.ndimension() == 3:
        face_tensor = face_tensor.unsqueeze(0)
    face_tensor = face_tensor.to(device)
    with torch.no_grad():
        emb = resnet(face_tensor).cpu().numpy()
    # L2-normalize
    emb = emb / np.linalg.norm(emb, axis=1, keepdims=True)
    return emb[0]

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [5]:

def load_db(path=DB_PATH):
    if os.path.exists(path):
        with open(path, 'rb') as f:
            db = pickle.load(f)
    else:
        db = {'users': {}, 'embeddings': {}, 'attendance': []}
    return db

def save_db(db, path=DB_PATH):
    with open(path, 'wb') as f:
        pickle.dump(db, f)

db = load_db()
print('Loaded DB: users=', len(db['users']))


Loaded DB: users= 0


In [6]:

from datetime import datetime
def enroll_user(name, email):
    print(f"Enrolling: {name} ({email}) — Upload one clear frontal face photo (JPEG/PNG).")
    uploaded = files.upload()  # Colab file picker
    if not uploaded:
        print("No file uploaded.")
        return
    # Take first uploaded file
    fname = list(uploaded.keys())[0]
    bytestr = uploaded[fname]
    img = img_from_bytes(bytestr)
    face_tensor = get_face_tensor(img)
    if face_tensor is None:
        print("No face detected. Please upload a photo with a single clear face.")
        return
    # get embedding
    emb = get_embedding(face_tensor)
    # create user
    user_id = str(uuid.uuid4())
    # save photo locally (optional)
    saved_fname = f'enroll_{user_id}.jpg'
    img.save(saved_fname)
    # update DB
    db = load_db()
    db['users'][user_id] = {
        'name': name,
        'email': email,
        'created_at': datetime.utcnow().isoformat(),
        'photo_filename': saved_fname
    }
    db['embeddings'][user_id] = emb.tolist()  # convert to list for pickling
    save_db(db)
    print("Enrollment successful. user_id:", user_id)

# Example usage:
# enroll_user("Vicky", "vicky@example.com")


In [7]:

from datetime import datetime

def recognize_upload(location_id='default'):
    print("Upload a selfie for attendance (JPEG/PNG):")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded.")
        return
    fname = list(uploaded.keys())[0]
    img = img_from_bytes(uploaded[fname])
    face_tensor = get_face_tensor(img)
    if face_tensor is None:
        print("No face detected. Try again.")
        return
    emb = get_embedding(face_tensor)
    # match
    db = load_db()
    best_user = None
    best_score = -1
    for uid, v in db['embeddings'].items():
        db_emb = np.array(v)
        score = cosine_similarity(emb, db_emb)
        if score > best_score:
            best_score = score
            best_user = uid
    THRESHOLD = 0.6  # tune this
    if best_user and best_score >= THRESHOLD:
        user = db['users'][best_user]
        # log attendance
        rec = {
            'timestamp': datetime.utcnow().isoformat(),
            'user_id': best_user,
            'name': user['name'],
            'email': user['email'],
            'location_id': location_id,
            'method': 'upload',
            'match_score': float(best_score)
        }
        db['attendance'].append(rec)
        save_db(db)
        print(f"Matched: {user['name']} (score={best_score:.3f}) — attendance logged.")
        return rec
    else:
        print(f"No match (best_score={best_score:.3f}).")
        return None

# Example: run cell then call recognize_upload()


In [8]:
from IPython.display import Javascript, display
from base64 import b64decode

def capture_webcam(filename='webcam_capture.jpg'):
    js = Javascript("""
    async function capture(){
      const div = document.createElement('div');
      const video = document.createElement('video');
      const btn = document.createElement('button');
      btn.textContent = 'Capture';
      div.appendChild(video);
      div.appendChild(btn);
      document.body.appendChild(div);
      const stream = await navigator.mediaDevices.getUserMedia({video:true});
      video.srcObject = stream;
      video.play();
      await new Promise(resolve => btn.onclick = resolve);
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      const data = canvas.toDataURL('image/jpeg');
      stream.getTracks().forEach(t => t.stop());
      div.remove();
      google.colab.kernel.invokeFunction('notebook._save_camera_image', [data], {});
    }
    capture();
    """)
    display(js)

from google.colab import output
def _save_camera_image(data_url):
    header, encoded = data_url.split(',', 1)
    binary = b64decode(encoded)
    with open('webcam_capture.jpg', 'wb') as f:
        f.write(binary)
    print('webcam_capture.jpg saved in Colab.')
output.register_callback('notebook._save_camera_image', _save_camera_image)

# After running the cell above, run:
# capture_webcam()
# Then use the saved file 'webcam_capture.jpg' for recognition (call recognize_file below)


In [9]:
def recognize_file(filepath='webcam_capture.jpg', location_id='default'):
    if not os.path.exists(filepath):
        print("File not found:", filepath)
        return
    img = Image.open(filepath).convert('RGB')
    face_tensor = get_face_tensor(img)
    if face_tensor is None:
        print("No face detected.")
        return
    emb = get_embedding(face_tensor)
    # matching logic same as above
    db = load_db()
    best_user = None
    best_score = -1
    for uid, v in db['embeddings'].items():
        db_emb = np.array(v)
        score = cosine_similarity(emb, db_emb)
        if score > best_score:
            best_score = score
            best_user = uid
    THRESHOLD = 0.6
    if best_user and best_score >= THRESHOLD:
        user = db['users'][best_user]
        rec = {
            'timestamp': datetime.utcnow().isoformat(),
            'user_id': best_user,
            'name': user['name'],
            'email': user['email'],
            'location_id': location_id,
            'method': 'webcam',
            'match_score': float(best_score)
        }
        db['attendance'].append(rec)
        save_db(db)
        print(f"Matched: {user['name']} (score={best_score:.3f}) — attendance logged.")
        return rec
    else:
        print(f"No match (best_score={best_score:.3f}).")
        return None


In [10]:
def export_attendance_csv(csv_path=ATTENDANCE_CSV):
    db = load_db()
    df = pd.DataFrame(db['attendance'])
    if df.empty:
        print("No attendance records yet.")
        return None
    df.to_csv(csv_path, index=False)
    print("Saved CSV:", csv_path)
    return csv_path

# To trigger download in browser:
# path = export_attendance_csv(); files.download(path)


In [11]:
def compute_score_stats():
    db = load_db()
    uids = list(db['embeddings'].keys())
    embs = [np.array(db['embeddings'][uid]) for uid in uids]
    # compute pairwise cosine similarities
    sims = []
    for i in range(len(embs)):
        for j in range(i+1, len(embs)):
            sims.append(cosine_similarity(embs[i], embs[j]))
    import numpy as np
    sims = np.array(sims)
    print("pairs:", len(sims))
    print("min, mean, max similarity:", sims.min(), sims.mean(), sims.max())
    return sims

# Use this after enrolling multiple users to see distribution.
